# **[HW24] NaiveBayes Classifier**
1. Requirements
2. Data Preprocessing
3. Model Training
4. Evaluation

## 1. Requirements

#### 1.1 필요한 패키지를 설치(install) 및 import 합니다.

In [1]:
# 한국어 전처리 라이브러리 
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 39.0 MB/s 


In [2]:
from tqdm import tqdm
from collections import defaultdict
import math

# POS(Part of Speech) tagger
from konlpy import tag 

#### 1.2 Train data 와 test data 를 준비합니다.

In [3]:
data = {}
# training data. input text 와 정답 label (긍정(1), 부정(0)) 로 구성.

data['train'] = [{'text': "정말 재미있습니다. 추천합니다."},
                {'text': "기대했던 것보단 별로였네요."},
                {'text': "지루해서 다시 보고 싶다는 생각이 안 드네요."},
                {'text': "완전 최고입니다 ! 다시 보고 싶습니다."},
                {'text': "연기도 연출도 다 만족스러웠습니다."},
                {'text': "연기가 좀 별로였습니다."},
                {'text': "연출도 좋았고 배우분들 연기도 최고입니다."},
                {'text': "기념일에 방문했는데 연기도 연출도 다 좋았습니다."},
                {'text': "전반적으로 지루했습니다. 저는 별로였네요."},
                {'text': "CG에 조금 더 신경 썼으면 좋겠습니다."}
                ]
# test data
data['test'] = [{'text': "최고입니다. 또 보고 싶네요."},
                {'text': "별로였습니다. 되도록 보지 마세요."},
                {'text': "다른 분들께 추천드릴 수 있을 만큼 연출도 연기도 만족했습니다."},
                {'text': "연기가 좀 더 개선되었으면 좋겠습니다."}
                ]

train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
test_labels = [1, 0, 1, 0]

### 2. Data Preprocessing


#### 2.1 한글 형태소 분석기를 이용해서 주어진 데이터를 tokenize 합니다.

오픈소스 형태소 분석기를 제공하는 파이썬 패키지 KoNLPy에서 제공하는 [꼬꼬마(Kkma) 형태소 분석기](https://konlpy.org/en/v0.5.2/api/konlpy.tag/#module-konlpy.tag._kkma)를 이용하여 tokenize 합니다.

In [4]:
# 형태소 분석기 선언
morph_analyzer = tag.Kkma() 

In [5]:
# tokenization 함수 정의
def tokenization(data, morph_analyzer):
    '''
    (input) data: list of data examples.
            morph_analyzer: morphological analyzer.
    (output) tokenized_data: list of tokenized data examples.
    '''
    tokenized_data = []

    for example in tqdm(data):
        tokens = morph_analyzer.morphs(example['text'])
        tokenized_data.append(tokens)

    return tokenized_data

In [6]:
# tokenization 함수를 이용한 데이터 tokenization
tokenized_data = {}

tokenized_data['train'] = tokenization(data['train'], morph_analyzer)
tokenized_data['test'] = tokenization(data['test'], morph_analyzer)

100%|██████████| 4/4 [00:00<00:00, 17.89it/s]


In [7]:
# tokenized_data 확인
tokenized_data['train']

[['정말', '재미있', '습니다', '.', '추천', '하', 'ㅂ니다', '.'],
 ['기대', '하', '었', '더', 'ㄴ', '것', '보다', 'ㄴ', '별', '로', '이', '었', '네요', '.'],
 ['지루', '하', '어서', '다시', '보', '고', '싶', '다는', '생각', '이', '안', '들', '네요', '.'],
 ['완전', '최고', '이', 'ㅂ니다', '!', '다시', '보', '고', '싶', '습니다', '.'],
 ['연기', '도', '연출', '도', '다', '만족', '스럽', '었', '습니다', '.'],
 ['연기', '가', '좀', '별', '로', '이', '었', '습니다', '.'],
 ['연출', '도', '좋', '았', '고', '배우', '분', '들', '연기', '도', '최고', '이', 'ㅂ니다', '.'],
 ['기념일',
  '에',
  '방문',
  '하',
  '었',
  '는데',
  '연기',
  '도',
  '연출',
  '도',
  '다',
  '좋',
  '았',
  '습니다',
  '.'],
 ['전반적',
  '으로',
  '지루',
  '하',
  '었',
  '습니다',
  '.',
  '저',
  '는',
  '별',
  '로',
  '이',
  '었',
  '네요',
  '.'],
 ['CG', '에', '조금', '더', '신경', '쓰', '었', '으면', '좋', '겠', '습니다', '.']]

#### 2.2 tokenization 결과를 이용해서 word to index dictionary 를 생성합니다.


In [8]:
# train data의 tokenization 결과에서 unique token만 남긴 set으로 변환
tokens = [token for i in range(len(tokenized_data['train'])) for token in tokenized_data['train'][i] ]
unique_train_tokens = set(tokens)

# NaiveBayes Classifier의 input에 들어갈 word의 index를 반환해주는 dictionary를 생성
word2index = defaultdict() # key: word, value: index of word
idx = 0
for token in tqdm(unique_train_tokens):
    word2index[token] = idx
    idx += 1

100%|██████████| 56/56 [00:00<00:00, 64017.72it/s]


### 3. Model Training

#### 3.1 NaiveBayes Classifier 모델 클래스를 구현합니다.


In [9]:
class NaiveBayesClassifier():
    def __init__(self, word2index, k=0.1):
        """
        (input) word2index: mapping a word to a pre-assigned index.
        """
        self.k = k # for smoothing
        self.word2index = word2index
        self.priors = {} # Prior probability for each class, P(c)
        self.likelihoods = {} # Likelihood for each token, P(d|c)

    def _set_priors(self, labels):
        """
        Set prior probability for each class, P(c).
        Count the number of each class and calculate P(c) for each class.
        """
        
        # Count the number of each class
        class_counts = defaultdict(int)
        for label in tqdm(labels):
            class_counts[label] += 1
        
        # For each class, calcuate P(c)
        for label, count in class_counts.items():
            self.priors[label] = class_counts[label] / len(labels)

    def _set_likelihoods(self, tokens, labels):
        """
        Set likelihood for each token, P(d|c).
        First, count the number of each class for each token.
        Then, calculate P(d|c) for a given class and token.
        """
        token_dists = {}
        number_of_token_for_class = defaultdict(int)

        for i, label in enumerate(tqdm(labels)):
            count = 0

            for token in tokens[i]:
                if token in self.word2index:
                    if token not in token_dists:
                        token_dists[token] = {0:0, 1:0}
                    token_dists[token][label] += 1
                count += 1
            number_of_token_for_class[label] += count
        print(token_dists)

        for token, dist in tqdm(token_dists.items()):
            if token not in self.likelihoods:
                self.likelihoods[token] = {
                    0: (token_dists[token][0]+ self.k) / (number_of_token_for_class[0] + len(self.word2index)* self.k),
                    1: (token_dists[token][1]+ self.k) / (number_of_token_for_class[1] + len(self.word2index)* self.k),
                }

    def train(self, input_tokens, labels):
        """
        (input) input_tokens: list of tokenized train data.
                labels: train labels for each sentence/document.
        """
        self._set_priors(labels)
        self._set_likelihoods(input_tokens, labels)

    def inference(self, input_tokens):
        """
        (input) input_tokens: list_of tokenized test data.
        """
        log_prob_0 = 0.0
        log_prob_1 = 0.0

        for token in input_tokens:
            if token in self.likelihoods:
                log_prob_0 += math.log(self.likelihoods[token][0])
                log_prob_1 += math.log(self.likelihoods[token][1])

        log_prob_0 += math.log(self.priors[0])
        log_prob_1 += math.log(self.priors[1])

        if log_prob_0 >= log_prob_1:
            return 0
        else:
            return 1

#### 3.2 주어진 학습 데이터에 대해 문장 분류 모델을 학습시킵니다.

In [10]:
# 문장 분류 모델 선언 및 학습
classifier = NaiveBayesClassifier(word2index)
classifier.train(tokenized_data['train'], train_labels)

100%|██████████| 10/10 [00:00<00:00, 32974.09it/s]


{'정말': {0: 0, 1: 1}, '재미있': {0: 0, 1: 1}, '습니다': {0: 3, 1: 4}, '.': {0: 6, 1: 6}, '추천': {0: 0, 1: 1}, '하': {0: 3, 1: 2}, 'ㅂ니다': {0: 0, 1: 3}, '기대': {0: 1, 1: 0}, '었': {0: 6, 1: 2}, '더': {0: 2, 1: 0}, 'ㄴ': {0: 2, 1: 0}, '것': {0: 1, 1: 0}, '보다': {0: 1, 1: 0}, '별': {0: 3, 1: 0}, '로': {0: 3, 1: 0}, '이': {0: 4, 1: 2}, '네요': {0: 3, 1: 0}, '지루': {0: 2, 1: 0}, '어서': {0: 1, 1: 0}, '다시': {0: 1, 1: 1}, '보': {0: 1, 1: 1}, '고': {0: 1, 1: 2}, '싶': {0: 1, 1: 1}, '다는': {0: 1, 1: 0}, '생각': {0: 1, 1: 0}, '안': {0: 1, 1: 0}, '들': {0: 1, 1: 1}, '완전': {0: 0, 1: 1}, '최고': {0: 0, 1: 2}, '!': {0: 0, 1: 1}, '연기': {0: 1, 1: 3}, '도': {0: 0, 1: 6}, '연출': {0: 0, 1: 3}, '다': {0: 0, 1: 2}, '만족': {0: 0, 1: 1}, '스럽': {0: 0, 1: 1}, '가': {0: 1, 1: 0}, '좀': {0: 1, 1: 0}, '좋': {0: 1, 1: 2}, '았': {0: 0, 1: 2}, '배우': {0: 0, 1: 1}, '분': {0: 0, 1: 1}, '기념일': {0: 0, 1: 1}, '에': {0: 1, 1: 1}, '방문': {0: 0, 1: 1}, '는데': {0: 0, 1: 1}, '전반적': {0: 1, 1: 0}, '으로': {0: 1, 1: 0}, '저': {0: 1, 1: 0}, '는': {0: 1, 1: 0}, 'CG': {0: 1, 1: 0},

100%|██████████| 56/56 [00:00<00:00, 156170.89it/s]


In [11]:
classifier.likelihoods

{'!': {0: 0.0014367816091954025, 1: 0.01729559748427673},
 '.': {0: 0.08764367816091954, 1: 0.09591194968553458},
 'CG': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 'ㄴ': {0: 0.030172413793103453, 1: 0.0015723270440251573},
 'ㅂ니다': {0: 0.0014367816091954025, 1: 0.04874213836477988},
 '가': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '것': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '겠': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '고': {0: 0.015804597701149427, 1: 0.0330188679245283},
 '기념일': {0: 0.0014367816091954025, 1: 0.01729559748427673},
 '기대': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '네요': {0: 0.04454022988505748, 1: 0.0015723270440251573},
 '는': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '는데': {0: 0.0014367816091954025, 1: 0.01729559748427673},
 '다': {0: 0.0014367816091954025, 1: 0.0330188679245283},
 '다는': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '다시': {0: 0.015804597701149427, 1: 0.01729559748427673},
 '

### 4. Evaluation

각각의 Test 데이터에 대해 정답값을 추론하고 Accuracy를 구합니다.

In [12]:
# Test 데이터 inference
preds = []
for test_tokens in tqdm(tokenized_data['test']):
    pred = classifier.inference(test_tokens)
    preds.append(pred)

100%|██████████| 4/4 [00:00<00:00, 19485.73it/s]


In [13]:
# Accuracy 측정
from sklearn.metrics import accuracy_score

print(accuracy_score(test_labels, preds))

1.0
